In [3]:
using JuMP;
using Clp;
using Ipopt;

N = 100;

# Just give an arbirtrary xval initially, will change iteratively anyway
xval = zeros(2);
Subprob = Model(solver=ClpSolver());
@variable(Subprob,y[1:2]>=0);
@objective(Subprob,Min,7*y[1]+12*y[2]);
# Give a reference to each constraint since we will use the dual multiplier associated with each constraint
# Just give an arbitrary rhs for now, will change iteratively anyway
@constraint(Subprob,constr1, y[1] >= 0);
@constraint(Subprob,constr2, y[2] >= 0);

xi1 = rand(-0.8:0.8, N);
xi2 = randn(N)*sqrt(12);
xi3 = zeros(N);
for i = 1:N
  xi3[i] = -2.5*log(rand());
end
xi4 = randn(N)*3;

# Level method
println("Begin solving the problem using level decomposition...");
singleCutMaster = Model(solver=ClpSolver());
@variable(singleCutMaster,x[1:2]>=0);
@variable(singleCutMaster,theta>=0);
@constraint(singleCutMaster, x[1]+x[2] <= 100);
@objective(singleCutMaster,Min,2*x[1]+3*x[2]+theta);

QPMaster = Model(solver=IpoptSolver());
@variable(QPMaster,x[1:2]>=0);
@variable(QPMaster,theta>=0);
@constraint(QPMaster, x[1]+x[2] <= 100);
@objective(QPMaster,Min,x[1]*x[1]+x[2]*x[2]);
@constraint(QPMaster, levelcon, 2*x[1]+3*x[2]+theta <= 2017);

level = -1;
LB = 0;
UB = 1e10;
tic();
iter = 0;
while (UB-LB)*1.0/UB > 1e-5
  iter = iter + 1;
  # First solve the master problem, and get x value
  if iter == 1
      status = solve(singleCutMaster);
      xval = getvalue(x);
      thetaval = getvalue(theta);
      LB = getobjectivevalue(singleCutMaster);
      # Just need to change the constraint, do not need to reconstruct the model
      duals = zeros(2);
      tempUB = 2*xval[1]+3*xval[2];
      x1coef = 0;
      x2coef = 0;
      rhs = 0;
      for k = 1:N
        JuMP.setRHS(constr1, 180+xi2[k]-6*xval[2]-(2+xi1[k])*xval[1]);
        JuMP.setRHS(constr2, 162+xi4[k]-3*xval[1]-(3.4-xi3[k])*xval[2]);
        # This is a simple case, since we have complete recourse: so there is no feasibility cuts
        status = solve(Subprob);
        tempUB = tempUB + getobjectivevalue(Subprob)*1.0/N;
        dual1 = getdual(constr1);
        dual2 = getdual(constr2);
        x1coef = x1coef + (2*dual1+xi1[k]*dual1+3*dual2);
        x2coef = x2coef + (6*dual1+3.4*dual2-xi3[k]*dual2);
        rhs = rhs + (180*dual1+xi2[k]*dual1+162*dual2+xi4[k]*dual2);
      end
      x1coef = x1coef*1.0/N;
      x2coef = x2coef*1.0/N;
      rhs = rhs*1.0/N;
      if tempUB < UB
        UB = tempUB;
      end
      level = LB+(UB-LB)*0.3;
      @constraint(QPMaster,theta+x1coef*x[1]+x2coef*x[2]>=rhs);
  else
      status = solve(QPMaster);
      if status == :Optimal
          xval = getvalue(x);
          thetaval = getvalue(theta);
          duals = zeros(2);
          tempUB = 2*xval[1]+3*xval[2];
          x1coef = 0;
          x2coef = 0;
          rhs = 0;
          for k = 1:N
            JuMP.setRHS(constr1, 180+xi2[k]-6*xval[2]-(2+xi1[k])*xval[1]);
            JuMP.setRHS(constr2, 162+xi4[k]-3*xval[1]-(3.4-xi3[k])*xval[2]);
            # This is a simple case, since we have complete recourse: so there is no feasibility cuts
            status = solve(Subprob);
            tempUB = tempUB + getobjectivevalue(Subprob)*1.0/N;
            dual1 = getdual(constr1);
            dual2 = getdual(constr2);
            x1coef = x1coef + (2*dual1+xi1[k]*dual1+3*dual2);
            x2coef = x2coef + (6*dual1+3.4*dual2-xi3[k]*dual2);
            rhs = rhs + (180*dual1+xi2[k]*dual1+162*dual2+xi4[k]*dual2);
          end
          x1coef = x1coef*1.0/N;
          x2coef = x2coef*1.0/N;
          rhs = rhs*1.0/N;
          if tempUB < UB
            UB = tempUB;
          end
          @constraint(QPMaster,theta+x1coef*x[1]+x2coef*x[2]>=rhs);
      else
          LB = level;
      end
      level = LB+(UB-LB)*0.3;  
  end
  if (UB-LB)*1.0/UB < 1e-5
    break;
  end
  JuMP.setRHS(levelcon, level);
end

println("LB = ", LB);
println("UB = ", UB);
println("xval = ", xval);
println("# of iterations = ", iter);
toc();
println("########################## Sample Size", N, "##########################");

Begin solving the problem using level decomposition...
This is Ipopt version 3.12.2, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        6
Number of nonzeros in Lagrangian Hessian.............:        2

Total number of variables............................:        3
                     variables with only lower bounds:        3
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:        3
        inequality constraints with only lower bounds:        1
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        2

iter    objective    inf_pr   in

This is Ipopt version 3.12.2, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:       18
Number of nonzeros in Lagrangian Hessian.............:        2

Total number of variables............................:        3
                     variables with only lower bounds:        3
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:        7
        inequality constraints with only lower bounds:        5
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        2

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0 1

This is Ipopt version 3.12.2, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:       24
Number of nonzeros in Lagrangian Hessian.............:        2

Total number of variables............................:        3
                     variables with only lower bounds:        3
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:        9
        inequality constraints with only lower bounds:        7
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        2

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0 1

This is Ipopt version 3.12.2, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:       30
Number of nonzeros in Lagrangian Hessian.............:        2

Total number of variables............................:        3
                     variables with only lower bounds:        3
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:       11
        inequality constraints with only lower bounds:        9
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        2

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0 1

This is Ipopt version 3.12.2, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:       30
Number of nonzeros in Lagrangian Hessian.............:        2

Total number of variables............................:        3
                     variables with only lower bounds:        3
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:       11
        inequality constraints with only lower bounds:        9
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        2

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0 1

...

This is Ipopt version 3.12.2, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:       42
Number of nonzeros in Lagrangian Hessian.............:        2

Total number of variables............................:        3
                     variables with only lower bounds:        3
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:       15
        inequality constraints with only lower bounds:       13
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        2

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0 1

This is Ipopt version 3.12.2, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:       45
Number of nonzeros in Lagrangian Hessian.............:        2

Total number of variables............................:        3
                     variables with only lower bounds:        3
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:       16
        inequality constraints with only lower bounds:       14
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        2

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0 1

This is Ipopt version 3.12.2, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:       48
Number of nonzeros in Lagrangian Hessian.............:        2

Total number of variables............................:        3
                     variables with only lower bounds:        3
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:       17
        inequality constraints with only lower bounds:       15
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        2

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0 1

This is Ipopt version 3.12.2, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:       51
Number of nonzeros in Lagrangian Hessian.............:        2

Total number of variables............................:        3
                     variables with only lower bounds:        3
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:       18
        inequality constraints with only lower bounds:       16
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        2

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0 1

This is Ipopt version 3.12.2, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:       51
Number of nonzeros in Lagrangian Hessian.............:        2

Total number of variables............................:        3
                     variables with only lower bounds:        3
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:       18
        inequality constraints with only lower bounds:       16
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        2

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0 1